In [23]:
import pandas as pd
import numpy as np
import requests
import json

In [24]:
df=pd.read_csv(r'C:\Users\飘哥\Desktop\数据爬取\OD数据.csv',encoding='gb2312')
df

,FSTR_ONAME,FSTR_DNAME,SLat,SLng,ELat,ELng
0,曹路,嘉定北,31.274748,121.677908,31.393509,121.233045
1,野生动物园,武宁路,31.052450,121.695519,31.236148,121.425952
2,江宁路,野生动物园,31.246219,121.439999,31.052450,121.695519
3,马当路,东方绿舟,31.212394,121.474834,31.100537,121.015184
4,金平路,豫园,31.013123,121.405411,31.230272,121.482826
...,...,...,...,...,...,...
14577,鹤沙航城,南陈路,31.080317,121.607581,31.323528,121.394245
14578,黄兴公园,淞发路,31.297807,121.528892,31.347176,121.496036
14579,黄陂南路,江杨北路,31.224733,121.468694,31.409674,121.435220
14580,龙柏新村,东兰路,31.179500,121.365743,31.157600,121.387340


In [25]:
def stod(slat,slng,dlat,dlng):#输入：起点纬度、起点经度、终点纬度、终点经度
    url ="http://api.map.baidu.com/direction/v2/transit?" #API地址
    ak = '' #秘钥
    #参数设置为wgs84坐标系和地铁优先
    real_url = url +"coord_type=wgs84&tactics_incity=5&origin="+str(slat)+","+str(slng)+"&destination="+str(dlat)+","+str(dlng)+"&ak="+ak #完整的请求代码
    req = requests.get(real_url)
    t = req.text
    data = json.loads(t) #将数据保存在数组data中
    try:#防止某几条数据报错导致请求终止
        total_distance= data['result']["routes"][0]["distance"]/1000
        total_duration = data['result']["routes"][0]["duration"]/60
    except:
        total_distance = None
        total_duration = None
    req.close()
    return total_distance,total_duration#返回数据为总距离，总时长


In [26]:
final=[]
for i in range(0,len(df)):
    result=stod(df.SLat[i],df.SLng[i],df.ELat[i],df.ELng[i])
    result2=pd.DataFrame(result).T
    final.append(result2)
    result2['FSTR_ONAME']=df.FSTR_ONAME[i]
    result2['FSTR_DNAME']=df.FSTR_DNAME[i]
#     final.append((result[0],result[1]))
final=pd.concat(final,0)
final

,0,1,FSTR_ONAME,FSTR_DNAME
0,57.609,103.233333,曹路,嘉定北
0,44.498,78.066667,野生动物园,武宁路
0,43.460,79.800000,江宁路,野生动物园
0,51.456,86.350000,马当路,东方绿舟
0,31.602,85.033333,金平路,豫园
...,...,...,...,...
0,41.787,78.666667,鹤沙航城,南陈路
0,16.780,44.383333,黄兴公园,淞发路
0,26.388,60.466667,黄陂南路,江杨北路
0,16.250,50.216667,龙柏新村,东兰路


In [27]:
final.to_csv(r'C:\Users\飘哥\Desktop\数据爬取\结果\OD1.csv',encoding='gb2312')